### Keras RNN으로 BBC 기사 분류

1. 패키지 수입 및 파라미터 설정

In [ ]:
# 패키지 수입
import csv
import numpy as np
import nltk

from keras.preprocessing.text import Tokenizer 
from keras.preprocessing.sequence import pad_sequences # 기사 길이를 맞출 때 사용 
from time import time
from sklearn.model_selection import train_test_split

from keras.models import Sequential
from keras.layers import Dense
from keras.layers import LSTM, Dropout, Embedding
from keras.layers import Bidirectional
from sklearn.metrics import confusion_matrix, f1_score

In [ ]:
# 하이퍼 파라미터 설정
MY_VOCAB = 5000     # 사용할 단어 수, 제일 많이 사용된 단어
MY_EMBED = 64       # 임베딩 차원
MY_HIDDEN = 100     # 뉴런의 개수, LSTM 셀의 규모
MY_LEN = 200        # 기사의 길이

MY_SPLIT = 0.8      # 학습용 데이터
MY_SAMPLE = 123     # 샘플 기사
MY_EPOCH = 100      # 반복 학습 수
TRAIN_MODE = 1      # 학습 모드와 평가 모드 선택, 학습된 데이터를 파일로 저장할 때 사용

2. 데이터 처리

In [ ]:
# 제외어(stopword) 설정 
nltk.download('stopwords')
MY_STOP = set(nltk.corpus.stopwords.words('english')) # 영어와 관련된 제외어를 불러옴

print('영어 제외어')
print(MY_STOP)
print('제외어 개수 :', len(MY_STOP))

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
영어 제외어
{'how', 'which', 'are', 'has', 'then', 'when', 'those', "hadn't", 'doing', 'having', 'at', 'its', 'what', 'ain', 'about', 'needn', 'ourselves', 'that', 'nor', 'until', 'wasn', 'y', 'between', "needn't", 'now', "don't", 'have', 'with', 'haven', 'in', 'was', 'do', "didn't", 'and', 'yourself', 'me', 'herself', 'because', 'it', "doesn't", 'won', 'myself', 'ours', 'against', "won't", "she's", 'been', 'above', 'both', 'a', "aren't", 'isn', 'by', "you'd", "you'll", 'some', 'you', 'were', 'hers', 'theirs', 'wouldn', 'himself', 'for', 'shouldn', 'after', 'up', 'again', 'through', 'all', 'm', "weren't", 'whom', 'd', 'they', 'once', 'further', 'the', 'here', 'o', 'there', 'any', 'couldn', 'below', 'an', 'than', 'few', 'him', 'during', 'where', 'aren', 'who', 'if', 'on', 'ma', 'or', 'hadn', 'i', 'each', 'same', "mustn't", 'yourselves', 'my', 'did', "haven't", 'itself', 'am', "could

In [ ]:
# 데이터 보관 창고
original = [] # 원본 기사
articles = [] # 제외어를 삭제한 기사
labels = [] # 카테고리

In [ ]:
# BBC 파일 읽고 처리
with open('/content/drive/MyDrive/Colab Notebooks/data/bbc-text.csv','r') as file:
    # 컬럼 이름 읽기
    reader = csv.reader(file)    
    next(reader) # 컬럼 명 다음의 데이터부터

    # 기사 하나(한 행)씩 처리
    for row in reader:
        # 카테고리 저장
        labels.append(row[0])
        
        # 원본 기사 저장
        original.append(row[1])

        # 제외어 삭제 하기
        news = row[1]
        for word in MY_STOP:
            mask = ' ' + word + ' ' 
            news = news.replace(mask, ' ') # news에 mask를 ' '로 replace한 내용 저장
        articles.append(news)

print('처리한 기사 수 :', len(articles))

처리한 기사 수 : 2225


In [ ]:
# 샘플 기사 출력
print('샘플 기사 원본')
print(original[MY_SAMPLE]) # MY_SAMPLE번 째 기사 확인
print(labels[MY_SAMPLE])
# 알파벳 수 : len(original[MY_SAMPLE])
print('총 단어 수 :', len(original[MY_SAMPLE].split())) # 디폴트는 공백

샘플 기사 원본
screensaver tackles spam websites net users are getting the chance to fight back against spam websites  internet portal lycos has made a screensaver that endlessly requests data from sites that sell the goods and services mentioned in spam e-mail. lycos hopes it will make the monthly bandwidth bills of spammers soar by keeping their servers running flat out. the net firm estimates that if enough people sign up and download the tool  spammers could end up paying to send out terabytes of data.   we ve never really solved the big problem of spam which is that its so damn cheap and easy to do   said malte pollmann  spokesman for lycos europe.  in the past we have built up the spam filtering systems for our users   he said   but now we are going to go one step further.    we ve found a way to make it much higher cost for spammers by putting a load on their servers.  by getting thousands of people to download and use the screensaver  lycos hopes to get spamming websites constantly r

In [ ]:
# 제외어 처리 결과
print('샘플 기사 제외어 삭제본')
print(articles[MY_SAMPLE])
print('총 단어 수 :', len(articles[MY_SAMPLE].split())) # 디폴트는 공백

샘플 기사 제외어 삭제본
screensaver tackles spam websites net users getting chance fight back spam websites  internet portal lycos made screensaver endlessly requests data sites sell goods services mentioned spam e-mail. lycos hopes make monthly bandwidth bills spammers soar keeping servers running flat out. net firm estimates enough people sign download tool  spammers could end paying send terabytes data.   never really solved big problem spam damn cheap easy   said malte pollmann  spokesman lycos europe.  past built spam filtering systems users   said   going go one step further.    found way make much higher cost spammers putting load servers.  getting thousands people download use screensaver  lycos hopes get spamming websites constantly running almost full capacity. mr pollmann said intention stop spam websites working subjecting much data cope with. said screensaver carefully written ensure amount traffic generated user overload web.  every single user contribute three four megabytes per d

In [ ]:
# Tokenizer 처리
A_token = Tokenizer(num_words=MY_VOCAB, # 데이터(단어) 몇 개 가지고 있나
                    oov_token='OOV') # 'Out Of Vocab'. 
                                     # 자주 쓰이는 5000개 단어보다 덜 쓰이는(어려운) 단어 처리를 어떻게 할 것 인가
                                     # 그 단어를 OOV로 설정하겠다

A_token.fit_on_texts(articles) # articles의 단어를 숫자로 바꾸기 위한 준비과정
A_tokenized = A_token.texts_to_sequences(articles) # 토큰 결과. 실제로 영어 단어를 숫자로 바꾸어 줌
# hash function

# 전환의 예
# 숫자 -> 단어
print(A_token.sequences_to_texts([[1]])) # 1이라는 숫자를 단어로 역전환 => OOV(누락된 단어)가 출력됨.
# 단어 -> 숫자
print(A_token.texts_to_sequences(['the'])) # 'the' 단어가 어떤 숫자로 변환? 인기도와 관계 x

['OOV']
[[1230]]


In [ ]:
# Token 처리 결과 출력
sample = A_tokenized[MY_SAMPLE]
print(sample) 

[3171, 1, 816, 878, 115, 136, 382, 347, 716, 28, 816, 878, 228, 1, 3172, 27, 3171, 1, 4869, 203, 569, 733, 1772, 126, 4024, 816, 260, 395, 3172, 700, 21, 1649, 3629, 2848, 2606, 1, 2325, 2550, 453, 2918, 570, 115, 63, 2291, 381, 7, 1160, 780, 1860, 2606, 11, 92, 1572, 1051, 1, 203, 281, 154, 1, 138, 364, 816, 1, 2225, 847, 2, 1, 1, 178, 3172, 139, 255, 1109, 816, 1, 726, 136, 2, 52, 60, 10, 818, 3790, 195, 41, 21, 56, 494, 245, 2606, 1363, 1, 2550, 382, 1021, 7, 780, 70, 3171, 3172, 700, 23, 1, 878, 3992, 453, 343, 322, 1394, 3, 1, 2, 3427, 583, 816, 878, 297, 1, 56, 203, 2296, 2403, 2, 3171, 2708, 1069, 660, 812, 1287, 3883, 1540, 1, 466, 224, 504, 1540, 1, 31, 96, 1, 681, 111, 2, 10, 1898, 912, 2, 381, 7, 1160, 1, 878, 11, 722, 256, 1, 1287, 224, 504, 111, 3172, 79, 70, 260, 395, 716, 28, 2, 3, 1, 4, 1605, 10, 823, 455, 158, 823, 455, 2, 569, 2179, 4024, 816, 260, 395, 891, 733, 1772, 126, 221, 3677, 569, 316, 86, 1051, 816, 260, 395, 3677, 23, 1, 1454, 681, 111, 415, 569, 3171, 760,

In [ ]:
# 기사 통계 내기
# 제외어 제외, 토큰 처리 후.
longest = max([len(x) for x in A_tokenized])
print('제일 긴 기사 :', longest)

shortest = min([len(x) for x in A_tokenized])
print('제일 짧은 기사 :', shortest)

print('총 단어 수 :', len(A_token.word_counts)) # A_token.word_counts : 단어가 사용된 횟수 출력
# 총 단어 수 n개 중 MY_VOCAB개만 기계학습에 사용

제일 긴 기사 : 2280
제일 짧은 기사 : 50
총 단어 수 : 29698


In [ ]:
# 기사 길이 맞추기 -> 모든 기사의 길이를 맞춰줌
A_tokenized = pad_sequences(A_tokenized,
                            maxlen=MY_LEN,
                            padding='post', # 200 단어보다 짧은 기사는 0으로 뒷부분 패딩 처리
                            truncating='post') # 200 단어보다 긴 기사는 뒷 부분 삭제 

# 기사 길이 확인
longest = max([len(x) for x in A_tokenized])
print('제일 긴 기사 :', longest)

shortest = min([len(x) for x in A_tokenized])
print('제일 짧은 기사 :', shortest)

제일 긴 기사 : 200
제일 짧은 기사 : 200


In [ ]:
# 라벨 tokenization
C_token = Tokenizer()
C_token.fit_on_texts(labels) # 변환 준비 과정
C_tokenized = C_token.texts_to_sequences(labels) # 카테고리를 숫자로 변환한 결과

# 전환의 예
print(C_token.word_index)
print(C_tokenized)

{'sport': 1, 'business': 2, 'politics': 3, 'tech': 4, 'entertainment': 5}
[[4], [2], [1], [1], [5], [3], [3], [1], [1], [5], [5], [2], [2], [3], [1], [2], [3], [1], [2], [4], [4], [4], [1], [1], [4], [1], [5], [4], [3], [5], [3], [4], [5], [5], [2], [3], [4], [5], [3], [2], [3], [1], [2], [1], [4], [5], [3], [3], [3], [2], [1], [3], [2], [2], [1], [3], [2], [1], [1], [2], [2], [1], [2], [1], [2], [4], [2], [5], [4], [2], [3], [2], [3], [1], [2], [4], [2], [1], [1], [2], [2], [1], [3], [2], [5], [3], [3], [2], [5], [2], [1], [1], [3], [1], [3], [1], [2], [1], [2], [5], [5], [1], [2], [3], [3], [4], [1], [5], [1], [4], [2], [5], [1], [5], [1], [5], [5], [3], [1], [1], [5], [3], [2], [4], [2], [2], [4], [1], [3], [1], [4], [5], [1], [2], [2], [4], [5], [4], [1], [2], [2], [2], [4], [1], [4], [2], [1], [5], [1], [4], [1], [4], [3], [2], [4], [5], [1], [2], [3], [2], [5], [3], [3], [5], [3], [2], [5], [3], [3], [5], [3], [1], [2], [3], [3], [2], [5], [1], [2], [2], [1], [4], [1], [4], [4], 

In [ ]:
# 데이터 4분할
C_tokenized = np.array(C_tokenized) # numpy로 변환
X_train, X_test, Y_train, Y_test = train_test_split(A_tokenized, #  입력 data
                                                    C_tokenized, # 출력 data
                                                    train_size=MY_SPLIT,
                                                    shuffle=False)

# 데이터 모양 확인 
print('학습용 입력 데이터 모양 :', X_train.shape)
print('학습용 출력 데이터 모양 :', Y_train.shape)

print('평가용 입력 데이터 모양 :', X_test.shape)
print('평가용 출력 데이터 모양 :', Y_test.shape)

학습용 입력 데이터 모양 : (1780, 200)
학습용 출력 데이터 모양 : (1780, 1)
평가용 입력 데이터 모양 : (445, 200)
평가용 출력 데이터 모양 : (445, 1)


3. 인공 신경망 구현

In [ ]:
# RNN 구현
model = Sequential()

model.add(Embedding(input_dim=MY_VOCAB, # MY_VOCAB개를
                    output_dim=MY_EMBED)) # MY_EMBED개로

model.add(Dropout(rate=0.5)) # Dropout : 임의의 뉴런의 출력을 일부로 0으로 만드는 작업
                             # 과적합(overfitting) 방지하기 위해 뉴런 몇 개를 죽임

model.add(Bidirectional(LSTM(units=MY_HIDDEN)))

model.add(Dense(units=6,
                activation='softmax'))

print('RNN 요약')
model.summary()

RNN 요약
Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding (Embedding)        (None, None, 64)          320000    
_________________________________________________________________
dropout (Dropout)            (None, None, 64)          0         
_________________________________________________________________
bidirectional (Bidirectional (None, 200)               132000    
_________________________________________________________________
dense (Dense)                (None, 6)                 1206      
Total params: 453,206
Trainable params: 453,206
Non-trainable params: 0
_________________________________________________________________


4. 인공 신경망 학습

In [ ]:
# RNN 학습
model.compile(optimizer='adam',
              loss='sparse_categorical_crossentropy',
              metrics=['acc'])

print(' 학습 시작')
begin = time()

model.fit(x=X_train,
          y=Y_train,
          epochs=MY_EPOCH,
          verbose=1)

end = time()
print('학습 시간 : {:.2f}'.format(end - begin))

 학습 시작
Epoch 1/100
56/56 [==============================] - 11s 25ms/step - loss: 1.6683 - acc: 0.2414
Epoch 2/100
56/56 [==============================] - 1s 23ms/step - loss: 1.3684 - acc: 0.4860
Epoch 3/100
56/56 [==============================] - 1s 23ms/step - loss: 0.6220 - acc: 0.7836
Epoch 4/100
56/56 [==============================] - 1s 22ms/step - loss: 0.2995 - acc: 0.9235
Epoch 5/100
56/56 [==============================] - 1s 22ms/step - loss: 0.2123 - acc: 0.9343
Epoch 6/100
56/56 [==============================] - 1s 23ms/step - loss: 0.2476 - acc: 0.9288
Epoch 7/100
56/56 [==============================] - 1s 23ms/step - loss: 0.0538 - acc: 0.9900
Epoch 8/100
56/56 [==============================] - 1s 23ms/step - loss: 0.0727 - acc: 0.9880
Epoch 9/100
56/56 [==============================] - 1s 23ms/step - loss: 0.0129 - acc: 0.9992
Epoch 10/100
56/56 [==============================] - 1s 23ms/step - loss: 0.0106 - acc: 0.9991
Epoch 11/100
56/56 [=====================

5. 인공 신경망 평가

In [ ]:
# RNN 평가
score = model.evaluate(X_test,
                       Y_test,
                       verbose=0)

print('최종 손실값 : {:.2f}'.format(score[0]))
print('최종 정확도 : {:.2f}'.format(score[1]))

최종 손실값 : 0.22
최종 정확도 : 0.96
